In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
from torchtext.legacy import data
import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
!pip install spacy --upgrade

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.0.6)


In [3]:
%%bash
python -m spacy download en
# python -m spacy download de

  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-06-19 09:50:02.458910: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
# spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

## DataSet from https://quoradata.quora.com/First-Quora-Dataset-Release-Question-Pairs

In [5]:
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

--2021-06-19 09:50:42--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.1.2, 151.101.65.2, 151.101.129.2, ...
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.1.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘quora_duplicate_questions.tsv’

quora_duplicate_que 100%[===================>]  55.48M   130MB/s    in 0.4s    

2021-06-19 09:50:43 (130 MB/s) - ‘quora_duplicate_questions.tsv’ saved [58176133/58176133]



In [6]:
import pandas as pd

quora_duplicate_ques_dataset = pd.read_csv("/content/quora_duplicate_questions.tsv", sep='\t')

print(f' Total number of records or dimentions : {quora_duplicate_ques_dataset.shape}')

 Total number of records or dimentions : (404290, 6)


In [14]:
quora_duplicate_ques_dataset = quora_duplicate_ques_dataset[quora_duplicate_ques_dataset['is_duplicate']==1]

In [15]:
quora_duplicate_ques_dataset.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1
15,15,31,32,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,1
16,16,33,34,What does manipulation mean?,What does manipulation means?,1
18,18,37,38,Why are so many Quora users posting questions ...,Why do people ask Quora questions which can be...,1
20,20,41,42,Why do rockets look white?,Why are rockets and boosters painted white?,1
29,29,59,60,How should I prepare for CA final law?,How one should know that he/she completely pre...,1


In [16]:
quora_duplicate_ques_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149263 entries, 5 to 404286
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            149263 non-null  int64 
 1   qid1          149263 non-null  int64 
 2   qid2          149263 non-null  int64 
 3   question1     149263 non-null  object
 4   question2     149263 non-null  object
 5   is_duplicate  149263 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 8.0+ MB


In [17]:
quora_duplicate_ques_dataset.describe()

,id,qid1,qid2,is_duplicate
count,149263.000000,149263.000000,149263.000000,149263.0
mean,200804.505839,166297.474706,166591.079142,1.0
std,116180.195223,148197.861544,148272.329781,0.0
min,5.000000,11.000000,12.000000,1.0
25%,100168.000000,40367.500000,40526.000000,1.0
50%,200448.000000,118410.000000,118114.000000,1.0
75%,301239.500000,266092.000000,266916.500000,1.0
max,404286.000000,537926.000000,537927.000000,1.0


In [18]:
quora_duplicate_ques_dataset.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [19]:
# quora_duplicate_ques_dataset.dropna(subset=['question1','question2'], inplace=True)
# quora_duplicate_ques_dataset.shape

In [20]:
# quora_duplicate_ques_dataset.isnull().sum()

In [21]:
# def tokenize_de(text):
#     """
#     Tokenizes German text from a string into a list of strings (tokens) and reverses it
#     """
#     return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [24]:
question1 = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

question2 = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

fields = [('question1', question1),('question2',question2)]

In [25]:
quora_duplicate_ques_dataset.reset_index(drop=True,  inplace=True)

In [27]:
quora_duplicate_ques_examples = [data.Example.fromlist([str(quora_duplicate_ques_dataset.question1[i]),str(quora_duplicate_ques_dataset.question2[i])],fields) for i in range(quora_duplicate_ques_dataset.shape[0])]

In [29]:
quora_duplicate_dataset= data.Dataset(quora_duplicate_ques_examples, fields)

In [30]:
(train_data, test_data) = quora_duplicate_dataset.split(split_ratio=[0.7,0.3], random_state=random.seed(SEED)) 

In [31]:
print(f"Toatal examples: {len(quora_duplicate_ques_examples)}")
print(f"Number of training examples: {len(train_data.examples)}")
# print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Toatal examples: 149263
Number of training examples: 104484
Number of testing examples: 44779


In [32]:
print(vars(train_data.examples[0]))

{'question1': ['what', 'are', 'the', 'most', 'intellectually', 'stimulating', 'movies', 'you', 'have', 'ever', 'seen', '?'], 'question2': ['what', 'are', 'the', 'most', 'intellectually', 'stimulating', 'films', 'you', 'have', 'ever', 'watched', '?']}


In [33]:
print(vars(test_data.examples[0]))

{'question1': ['can', 'my', 'girlfriend', 'track', 'my', 'phone', 'through', 'google', 'if', 'we', 'have', 'same', 'google', 'account', '?'], 'question2': ['can', 'my', 'girl', 'friend', 'track', 'my', 'phone', 'and', 'read', 'my', 'text', 'messages', 'if', 'we', 'have', 'the', 'same', 'google', 'acount']}


In [34]:
question1.build_vocab(train_data, min_freq = 2)
question2.build_vocab(train_data, min_freq = 2)

In [35]:
print(f"Unique tokens in source vocabulary: {len(question1.vocab)}")
print(f"Unique tokens in target vocabulary: {len(question2.vocab)}")

Unique tokens in source vocabulary: 14513
Unique tokens in target vocabulary: 14455


In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
BATCH_SIZE = 128

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key= lambda x: len(x.question1),
    sort_within_batch=True, 
    device = device)

In [38]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [39]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [40]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [41]:
INPUT_DIM = len(question1.vocab)
OUTPUT_DIM = len(question2.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [42]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(14513, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(14455, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=14455, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [43]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 22,187,639 trainable parameters


In [44]:
optimizer = optim.Adam(model.parameters())

In [45]:
TRG_PAD_IDX = question2.vocab.stoi[question2.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [46]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.question1
        trg = batch.question2
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [47]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.question1
            trg = batch.question2

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [48]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [49]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 4m 16s
	Train Loss: 4.820 | Train PPL: 123.977
	 Val. Loss: 4.673 |  Val. PPL: 106.980
Epoch: 02 | Time: 4m 17s
	Train Loss: 3.830 | Train PPL:  46.043
	 Val. Loss: 4.163 |  Val. PPL:  64.232
Epoch: 03 | Time: 4m 17s
	Train Loss: 3.293 | Train PPL:  26.912
	 Val. Loss: 3.874 |  Val. PPL:  48.157
Epoch: 04 | Time: 4m 18s
	Train Loss: 2.970 | Train PPL:  19.499
	 Val. Loss: 3.765 |  Val. PPL:  43.184
Epoch: 05 | Time: 4m 20s
	Train Loss: 2.726 | Train PPL:  15.278
	 Val. Loss: 3.664 |  Val. PPL:  39.021
Epoch: 06 | Time: 4m 19s
	Train Loss: 2.542 | Train PPL:  12.705
	 Val. Loss: 3.625 |  Val. PPL:  37.537
Epoch: 07 | Time: 4m 17s
	Train Loss: 2.405 | Train PPL:  11.076
	 Val. Loss: 3.620 |  Val. PPL:  37.348
Epoch: 08 | Time: 4m 16s
	Train Loss: 2.298 | Train PPL:   9.957
	 Val. Loss: 3.566 |  Val. PPL:  35.364
Epoch: 09 | Time: 4m 17s
	Train Loss: 2.193 | Train PPL:   8.963
	 Val. Loss: 3.570 |  Val. PPL:  35.524
Epoch: 10 | Time: 4m 16s
	Train Loss: 2.118 | Train PPL

In [50]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.566 | Test PPL:  35.364 |
